# imports

In [ ]:
# imports
from niftypet import nipet
from niftypet import nimpa

from os import path
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as ipyw

import logging
logging.basicConfig(level=logging.DEBUG)

print(nimpa.gpuinfo())

# get all the scanner parameters
mMRpars = nipet.get_mmrparams()

# load

In [ ]:
folderin = "amyloidPET_FBP_TP0"
# automatically categorise the input data
logging.getLogger().setLevel(logging.INFO)
datain = nipet.classify_input(folderin, mMRpars)

# output path
opth = path.join(datain['corepath'], 'niftyout')
# switch on verbose mode
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
datain

In [ ]:
# obtain the hardware mu-map (the bed and the head&neck coil)
muhdct = nipet.hdw_mumap(datain, [1,2,4], mMRpars, outpath=opth, use_stored=True)

In [ ]:
# obtain the MR-based human mu-map
muodct = nipet.obj_mumap(datain, mMRpars, outpath=opth, store=True)

# debug plots

In [ ]:
plt.figure()
def plot_image_with_colour_bar(iz=60):
    """iz  : int, axial index"""
    try:  # needs HW maps
        im = muhdct['im'][iz,:,:] + muodct['im'][iz,:,:]
    except:
        im = muodct['im'][iz,:,:]
    plt.clf()
    plt.imshow(im, cmap='bone')
    plt.colorbar()
    plt.show()

ipyw.interact(plot_image_with_colour_bar,
    iz=ipyw.IntSlider(min=0, max=len(muodct['im']), step=1, value=len(muodct['im'])//2));

In [ ]:
hst = nipet.mmrhist(datain, mMRpars)

In [ ]:
# sinogram index (<127 for direct sinograms, >=127 for oblique sinograms)
si = 60

plt.matshow(hst['psino'][si,:,:], cmap='inferno')
plt.xlabel('bins')
plt.ylabel('angles')
plt.title('prompt sinogram')

plt.matshow(hst['dsino'][si,:,:], cmap='inferno')
plt.xlabel('bins')
plt.ylabel('angles')
plt.title('delayed sinogram');

# recon

In [ ]:
# OSEM 14 subsets
recon = nipet.mmrchain(
    datain, mMRpars,
    frames=['timings', [3000, 3600]],
    mu_h=muhdct,
    mu_o=muodct,
    itr=4,
    fwhm=0.0,
    outpath=opth,
    fcomment='niftypet-recon',
    store_img=True)

In [ ]:
plt.figure()
def plot_image_with_colour_bar(z=60):
    """z  : int, slice index"""
    plt.clf()
    ROI = slice(100, -100), slice(100, -100)
    plt.imshow(recon['im'][z][ROI], cmap='magma')
    plt.colorbar()
    plt.show()

ipyw.interact(plot_image_with_colour_bar,
    z=ipyw.IntSlider(min=0, max=len(recon['im']) - 1, step=1, value=len(recon['im'])//2));